Run the following code before running this script

ROAST to get Bvals

0.8_extract_human_data.ipynb

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_percentage_error as mape
from sklearn.metrics import mean_absolute_error as mae
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score

#
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Flatten, Dense


from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
#from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from scikeras.wrappers import KerasRegressor
from sklearn.ensemble import AdaBoostRegressor

import pickle
from keras.models import load_model

# Load prepared data

In [2]:
dir_path = "/Users/chikakoolsen/opt/python/thesis/code/tdcs_thesis/"
# dir_path = "/Users/mriworkshop/Documents/TDCS/code/tdcs_thesis/"
save_path = dir_path+"data/raw/"
img_path =  dir_path+"data/processed/"
model_path = dir_path+"models/"

## 3 attributes

In [3]:
file_mean = save_path+"fmap_mean_humans.txt"
columns_mean =['exp', 'mini_exp', 'i', 'j', 'k', 'neg', 'mean0', 'mean1', 'mean2']
data = np.loadtxt(file_mean);

In [26]:
data

array([[106.,   1.,   0., ...,   0.,   0.,   0.],
       [106.,   1.,   0., ...,   0.,   0.,   0.],
       [106.,   1.,   0., ...,   0.,   0.,   0.],
       ...,
       [401.,   3.,  43., ...,   0.,   0.,   0.],
       [401.,   3.,  43., ...,   0.,   0.,   0.],
       [401.,   3.,  43., ...,   0.,   0.,   0.]])

In [4]:
df = pd.DataFrame(data, columns=columns_mean)
df = df.astype({"exp": str, "i": int, "j": int, "k": int, "mini_exp": int, "neg": int})
df

,exp,mini_exp,i,j,k,neg,mean0,mean1,mean2
0,106.0,1,0,0,0,0,0.0,0.0,0.0
1,106.0,1,0,0,1,0,0.0,0.0,0.0
2,106.0,1,0,0,2,0,0.0,0.0,0.0
3,106.0,1,0,0,3,0,0.0,0.0,0.0
4,106.0,1,0,0,4,0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
3964923,401.0,3,43,63,59,1,0.0,0.0,0.0
3964924,401.0,3,43,63,60,1,0.0,0.0,0.0
3964925,401.0,3,43,63,61,1,0.0,0.0,0.0
3964926,401.0,3,43,63,62,1,0.0,0.0,0.0


In [28]:
df.loc[(df['i']==10)&(df['j']==3)&(df['k']==10)]

,exp,mini_exp,i,j,k,neg,mean0,mean1,mean2
41162,106.0,1,10,3,10,0,0.0,0.0,0.0
221386,106.0,1,10,3,10,1,0.0,0.0,0.0
401610,106.0,2,10,3,10,0,0.0,0.0,0.0
581834,106.0,2,10,3,10,1,0.0,0.0,0.0
762058,106.0,3,10,3,10,0,0.0,0.0,0.0
942282,106.0,3,10,3,10,1,0.0,0.0,0.0
1122506,106.0,4,10,3,10,0,0.0,0.0,0.0
1302730,106.0,4,10,3,10,1,0.0,0.0,0.0
1482954,106.0,5,10,3,10,0,0.0,0.0,0.0
1663178,106.0,5,10,3,10,1,0.0,0.0,0.0


In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3964928 entries, 0 to 3964927
Data columns (total 9 columns):
 #   Column    Dtype  
---  ------    -----  
 0   exp       object 
 1   mini_exp  int64  
 2   i         int64  
 3   j         int64  
 4   k         int64  
 5   neg       int64  
 6   mean0     float64
 7   mean1     float64
 8   mean2     float64
dtypes: float64(3), int64(5), object(1)
memory usage: 272.3+ MB


# Split data

## Get one experiment

In [5]:
df_test1 = df[(df['exp']=='303.0') & (df['mini_exp']==1) & (df['neg']==1)]
df_test2 = df[(df['exp']=='303.0') & (df['mini_exp']==1) & (df['neg']==0)]
df_test3 = df[(df['exp']=='303.0') & (df['mini_exp']==2) & (df['neg']==1)]
df_test4 = df[(df['exp']=='303.0') & (df['mini_exp']==2) & (df['neg']==0)]
# df_test5 = df[(df['exp']=='401.0') & (df['mini_exp']==3) & (df['neg']==1)]
# df_test6 = df[(df['exp']=='401.0') & (df['mini_exp']==3) & (df['neg']==0)]

In [163]:
print(df_test1.shape)
print(df_test2.shape)
print(df_test3.shape)
print(df_test4.shape)
print(df_test5.shape)
print(df_test6.shape)

(180224, 9)
(180224, 9)
(180224, 9)
(180224, 9)
(180224, 10)
(180224, 10)


In [106]:
df_test1.iloc[:, 6:]

,mean0,mean1,mean2
3063808,0.0,0.0,0.0
3063809,0.0,0.0,0.0
3063810,0.0,0.0,0.0
3063811,0.0,0.0,0.0
3063812,0.0,0.0,0.0
...,...,...,...
3244027,0.0,0.0,0.0
3244028,0.0,0.0,0.0
3244029,0.0,0.0,0.0
3244030,0.0,0.0,0.0


In [167]:
X_pred1 = df_test1.iloc[:, 6:].values 
# y_pred1 = df_test1['theory'].values

X_pred2 = df_test2.iloc[:, 6:].values 
# y_pred2 = df_test2['theory'].values

X_pred3 = df_test3.iloc[:, 6:].values 
# y_pred3 = df_test3['theory'].values

X_pred4 = df_test4.iloc[:, 6:].values 
# y_pred4 = df_test4['theory'].values

# X_pred5 = df_test5.iloc[:, 6:].values 
# # y_pred5 = df_test5['theory'].values

# X_pred6 = df_test6.iloc[:, 6:].values 
# # y_pred6 = df_test6['theory'].values

In [6]:
df_test1

,exp,mini_exp,i,j,k,neg,mean0,mean1,mean2
2342912,303.0,1,0,0,0,1,0.0,0.0,0.0
2342913,303.0,1,0,0,1,1,0.0,0.0,0.0
2342914,303.0,1,0,0,2,1,0.0,0.0,0.0
2342915,303.0,1,0,0,3,1,0.0,0.0,0.0
2342916,303.0,1,0,0,4,1,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
2523131,303.0,1,43,63,59,1,0.0,0.0,0.0
2523132,303.0,1,43,63,60,1,0.0,0.0,0.0
2523133,303.0,1,43,63,61,1,0.0,0.0,0.0
2523134,303.0,1,43,63,62,1,0.0,0.0,0.0


## None zero

In [165]:
# 3 attributes
df_nonzero = df[(df['mean0']!=0.0) & (df['mean1']!=0.0) & (df['mean2']!=0.0)]

In [109]:
df_nonzero

,exp,mini_exp,i,j,k,neg,mean0,mean1,mean2
2083,106.0,1,0,32,35,0,-1509.636841,-5565.706543,-1693.869995
2084,106.0,1,0,32,36,0,-7155.602051,-1473.945801,-1987.783325
2148,106.0,1,0,33,36,0,0.000024,0.000024,0.000024
2149,106.0,1,0,33,37,0,0.000024,0.000024,6217.434082
2206,106.0,1,0,34,30,0,730.194763,644.190369,709.672607
...,...,...,...,...,...,...,...,...,...
3963417,401.0,3,43,40,25,1,-942.967896,-970.626160,-905.370056
3963418,401.0,3,43,40,26,1,-602.453796,-682.196106,-801.513672
3963419,401.0,3,43,40,27,1,-456.123444,-518.171326,-246.045349
3963477,401.0,3,43,41,21,1,-0.000012,-0.000012,-0.000012


In [166]:
df4_pred1 =  df_nonzero[(df_nonzero['exp']=='303.0') & (df_nonzero['mini_exp']==1) & (df_nonzero['neg']==1)]
df4_pred2 =  df_nonzero[(df_nonzero['exp']=='303.0') & (df_nonzero['mini_exp']==1) & (df_nonzero['neg']==0)]
df4_pred3 =  df_nonzero[(df_nonzero['exp']=='303.0') & (df_nonzero['mini_exp']==2) & (df_nonzero['neg']==1)]
df4_pred4 =  df_nonzero[(df_nonzero['exp']=='303.0') & (df_nonzero['mini_exp']==2) & (df_nonzero['neg']==0)]
# df4_pred5 =  df_nonzero[(df_nonzero['exp']=='401.0') & (df_nonzero['mini_exp']==3) & (df_nonzero['neg']==1)]
# df4_pred6 =  df_nonzero[(df_nonzero['exp']=='401.0') & (df_nonzero['mini_exp']==3) & (df_nonzero['neg']==0)]

In [111]:
df4_pred1.iloc[:, 6:]

,mean0,mean1,mean2
3065495,26.726738,-2.483254,-12.536347
3065496,-583.888184,-529.000488,-555.799438
3065499,-635.242188,-626.604858,-661.435425
3065547,-1606.291870,-1697.073242,-1734.718506
3065548,-1751.929443,-1800.989258,-1808.521118
...,...,...,...
3242522,-559.640625,-580.239014,-409.822723
3242523,-449.633636,-418.433075,-266.756378
3242578,-0.000013,-0.000012,-0.000012
3242581,-0.000012,-0.000012,-0.000012


In [168]:
X4_pred1 = df4_pred1.iloc[:, 6:].values
# y4_pred1 = df4_pred1['theory'].values

X4_pred2 = df4_pred2.iloc[:, 6:].values
# y4_pred2 = df4_pred2['theory'].values

X4_pred3 = df4_pred3.iloc[:, 6:].values
# y4_pred3 = df4_pred3['theory'].values

X4_pred4 = df4_pred4.iloc[:, 6:].values
# X4_pred5 = df4_pred5.iloc[:, 6:].values
# X4_pred6 = df4_pred6.iloc[:, 6:].values

In [169]:
print(X4_pred1.shape)
print(X4_pred2.shape)
print(X4_pred3.shape)
print(X4_pred4.shape)
# print(X4_pred5.shape)
# print(X4_pred6.shape)

(63553, 3)
(63595, 3)
(63513, 3)
(63472, 3)


## Load model

In [44]:
# 3 attributes
file = model_path+'best_model_nn_optimized.h5'
model = load_model(file)

2023-04-30 23:49:12.614543: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Predict

In [194]:
df_test = df_test4
df_pred = df4_pred4

In [195]:
X_test = X4_pred4

In [196]:
test_pred = model.predict(X_test)

1984/1984 [==============================] - 1s 741us/step


In [197]:
df_pred['predict'] = test_pred

/var/folders/yd/vkjb_lqj2_30lnyhzhjqtg9w0000gp/T/ipykernel_55574/255584968.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pred['predict'] = test_pred


In [198]:
df_test['predict'] = 0.00

/var/folders/yd/vkjb_lqj2_30lnyhzhjqtg9w0000gp/T/ipykernel_55574/3257651465.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test['predict'] = 0.00


In [199]:
# 3 attributes
df_out = df_test[['i', 'j', 'k', 'neg', 'predict']]
df_pre = df_pred[['i', 'j', 'k', 'neg', 'predict']]

In [200]:
# 3 attributes
for x in range(len(df_pre)):
    i = df_pre.iloc[x, :]['i'].astype(int)
    j = df_pre.iloc[x, :]['j'].astype(int)
    k = df_pre.iloc[x, :]['k'].astype(int)
    neg = df_pre.iloc[x, :]['neg'].astype(int)
    pred = df_pre.iloc[x, :]['predict']
    idx = df_out[(df_out['i']==i) & (df_out['j']==j) & (df_out['k']==k) & (df_out['neg']==neg)].index
    df_out.loc[idx, 'predict']= pred

In [201]:
np.savetxt(img_path+"nn_nonzero_best_cly_0303_2_pos.txt", df_out[['i', 'j', 'k', 'predict']], fmt="%i %i %i %s")